In [92]:
from polars import DataFrame, col
from random import randrange

# Time Weighted Return calcilation
[wiki](https://en.wikipedia.org/wiki/Time-weighted_return)


In [75]:
TX = DataFrame(
    {
       "id": [1, 2, 3],
        "timestamp": [3, 5, 7],
        "price": [9, 10, 11],
        "qty": [5, -3, 2],  # +buy, -sell
    }
)
print(len(TX))
TX = TX.extend(DataFrame({
    "id": [TX["id"].max() + 1],
    "timestamp": [TX["timestamp"].max() + 1],
    "price": [13],
    "qty": [TX["qty"].last()],  # +buy, -sell
})).sort("id")
print(len(TX))


Q = DataFrame(
    {
        "timestamp": range(10),
        "price": [randrange(9, 12) for _ in range(10)],
    }
)
QQ = Q.join_asof(TX, on="timestamp", strategy="backward", suffix=".tx")
QQ

3
4


timestamp,price,id,price.tx,qty
i64,i64,i64,i64,i64
0,9,null,null,null
1,11,null,null,null
2,11,null,null,null
3,10,1,9,5
4,10,1,9,5
5,11,2,10,-3
6,10,2,10,-3
7,10,3,11,2
8,11,4,13,2


In [79]:
AGR = QQ.group_by("id", maintain_order=True) \
    .agg(
        ts_start=col("timestamp").first(),
        ts_end=col("timestamp").last(),
        price_start=col("price").first(),
        price_end=col("price").last(),
        qty=col("qty").mode().first().fill_null(0),
    ).with_columns(
        price_diff=col("price_end") - col("price_start"),
        acc_qty=col("qty").rolling_sum(len(QQ), min_periods=1),
    ).with_columns(
        return_pct=col("price_diff") / col("price_start"),
        pnl=col("price_diff") * col("acc_qty")
    )

AGR

/var/folders/5d/vxh2t4ys5p5_2br33b1v3jr80000gn/T/ipykernel_81073/1338024783.py:10: DeprecationWarning: the argument `min_periods` for `Expr.rolling_sum` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  acc_qty=col("qty").rolling_sum(len(QQ), min_periods=1),


id,ts_start,ts_end,price_start,price_end,qty,price_diff,acc_qty,return_pct,pnl
i64,i64,i64,i64,i64,i64,i64,i64,f64,i64
null,0,2,9,11,0,2,0,0.222222,0
1,3,4,10,10,5,0,5,0.0,0
2,5,6,11,10,-3,-1,2,-0.090909,-2
3,7,7,10,10,2,0,4,0.0,0
4,8,9,11,10,2,-1,6,-0.090909,-6


In [ ]:
# filter out nulls
AGR.filter(col('id').is_not_null()) \
  .select(
    TWR=(col('return_pct')+1).product()-1, #TWR
    RET=col('pnl').sum(),
  )

TWR,RET
f64,i64
-0.173554,-8
